In [239]:
import pandas as pd
from collections import defaultdict
import networkx as nx
import os
import re
import matplotlib.pyplot as plt
from rapidfuzz import fuzz
import itertools

In [240]:
absolute_path = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\board_analysis\\"
altered_dataframes = "altered_dataframes\\"
gpt_dataframes = "gpt_dataframes\\"
graphs = "graphs\\"       # (Not used for saving in this version)
scripts = "scripts\\"
board_dataframes = "board_dataframes\\"
yearly_interlocks = "yearly_interlocks\\"

years = ["1999", "2000", "2005", "2007", "2008", "2009", "2011", "2013", "2018"]

In [241]:
def remove_non_samples(df: pd.DataFrame) -> pd.DataFrame:
    """Filter the DataFrame to include only rows where 'PrimarySample' is True."""
    return df[df['PrimarySample'] == True]

def normalize(x):
    """Convert the input to a string and strip whitespace."""
    return str(x).strip()
    


In [242]:
# ======================
# --- UNION–FIND SETUP ---
# ======================
# We'll use union–find to group identifiers (Institution names and AffiliationIds)
parent = {}
appearances = {}   # Map: normalized node -> set of years it appears in.
node_sources = {}  # Map: normalized node -> set of source tags ("Institution", "AffiliationId")

def find(x):
    x = normalize(x)
    if parent[x] != x:
        parent[x] = find(parent[x])
    return parent[x]

def union(x, y):
    x = normalize(x)
    y = normalize(y)
    rootX = find(x)
    rootY = find(y)
    if rootX != rootY:
        parent[rootY] = rootX

def add_node(node, source, year):
    node = normalize(node)
    if node == "":
        return
    if node not in parent:
        parent[node] = node
    appearances.setdefault(node, set()).add(year)
    node_sources.setdefault(node, set()).add(source)

# ----------------------------
# Record associations between AffiliationId and Institution.
# For rows where both values are present, we record that this affiliation id
# is paired with the institution name.
aff_to_inst = defaultdict(set)

# ======================
# --- MERGE THE DATA ---
# ======================
# We'll store all the boards in one DataFrame.
merged_boards_list = []  # to hold DataFrames for each year

for year in years:
    file_path = f"{absolute_path}{board_dataframes}{year}_boards.csv"
    boards_df = pd.read_csv(file_path)
    boards_df = remove_non_samples(boards_df)
    boards_df['Year'] = year  # add a Year column
    merged_boards_list.append(boards_df)
    
    # Process each row for union–find
    for _, row in boards_df.iterrows():
        inst = row.get("Institution")
        aff = row.get("AffiliationId")
        
        valid_inst = pd.notna(inst) and normalize(inst) != ""
        valid_aff  = pd.notna(aff)  and normalize(aff)  != ""
        
        # If Institution is present, record it.
        if valid_inst:
            add_node(inst, "Institution", year)
        
        # For AffiliationId:
        # - If only AffiliationId is present, record it.
        # - If both are present, record both and union them.
        if valid_aff:
            if not valid_inst:
                add_node(aff, "AffiliationId", year)
            else:
                add_node(aff, "AffiliationId", year)
                aff_to_inst[normalize(aff)].add(normalize(inst))
                union(inst, aff)

# Concatenate all boards into a single DataFrame.
merged_df = pd.concat(merged_boards_list, ignore_index=True)

# ======================
# --- GROUP INTO COMPONENTS ---
# ======================
components = {}
for node in parent:
    root = find(node)
    if root not in components:
        components[root] = {"nodes": set(), "years": set(), "sources": set()}
    components[root]["nodes"].add(node)
    if node in appearances:
        components[root]["years"].update(appearances[node])
    if node in node_sources:
        components[root]["sources"].update(node_sources[node])

# ======================
# --- IDENTIFY DISCREPANCY GROUPS ---
# ======================
# A group is flagged if:
#   1) It has more than one distinct Institution name (nodes from "Institution"), or
#   2) The group appears in some year(s) with only an AffiliationId (i.e. no direct Institution).
discrepancy_groups = set()
for comp_id, data in components.items():
    # Get all nodes that came from the "Institution" column.
    inst_nodes = {node for node in data['nodes'] if "Institution" in node_sources.get(node, set())}
    # Skip groups that never had any direct Institution data.
    if not inst_nodes:
        continue
    # Get the years where a direct Institution was reported.
    direct_inst_years = set()
    for node in inst_nodes:
        direct_inst_years.update(appearances.get(node, set()))
    # If more than one Institution name appears or there are years with only an AffiliationId, flag it.
    if (len(inst_nodes) > 1) or (data['years'] != direct_inst_years):
        discrepancy_groups.add(comp_id)

# ======================
# --- ASSIGN GROUP IDs TO THE MERGED DATAFRAME ---
# ======================
def get_group_id(row):
    # Use the nonempty field to compute the group.
    if pd.notna(row.get("Institution")) and normalize(row.get("Institution")) != "":
        return find(normalize(row.get("Institution")))
    elif pd.notna(row.get("AffiliationId")) and normalize(row.get("AffiliationId")) != "":
        return find(normalize(row.get("AffiliationId")))
    else:
        return None

merged_df["GroupID"] = merged_df.apply(get_group_id, axis=1)

# ======================
# --- DETERMINE THE CANONICAL INSTITUTION NAME FOR EACH DISCREPANT GROUP ---
# ======================
# For each discrepancy group, choose the institution name that appears most frequently
# (i.e. the mode among nonempty Institution entries in that group).
group_to_canonical = {}
for group_id in discrepancy_groups:
    group_rows = merged_df[merged_df["GroupID"] == group_id]
    # Only consider rows with a nonempty Institution value.
    valid_insts = group_rows["Institution"].dropna().apply(normalize)
    valid_insts = valid_insts[valid_insts != ""]
    if not valid_insts.empty:
        canonical_name = valid_insts.value_counts().idxmax()
        group_to_canonical[group_id] = canonical_name
        print(f"Group '{group_id}': replacing with canonical Institution '{canonical_name}'")
    else:
        # In the unlikely case no row has an Institution value, leave it as is.
        group_to_canonical[group_id] = None

# ======================
# --- UPDATE THE MERGED DATAFRAME ---
# ======================
# For every row that belongs to a discrepant group, replace its Institution column
# with the canonical name computed.
def update_institution(row):
    group_id = row["GroupID"]
    if pd.notna(group_id) and group_id in group_to_canonical and group_to_canonical[group_id]:
        return group_to_canonical[group_id]
    else:
        return row.get("Institution")

merged_df["Institution"] = merged_df.apply(update_institution, axis=1)

# Optionally drop the GroupID column if you no longer need it.
merged_df = merged_df.drop(columns=["GroupID"])
merged_df = merged_df.drop(columns=["FullName"])

# ======================
# --- WRITE THE MERGED DATAFRAME TO CSV ---
# ======================
# Ensure the output folder exists.
output_folder = "merged_boards"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "merged_boards.csv")
merged_df.to_csv(output_path, index=False)
print(f"\nMerged boards saved to: {output_path}")


Group 'University At Buffalo State University Of New York': replacing with canonical Institution 'University At Buffalo State University Of New York'
Group 'Virginia Tech': replacing with canonical Institution 'Virginia Polytechnic Institute And State University'
Group 'New School': replacing with canonical Institution 'New School'
Group 'Southern Illinois University System': replacing with canonical Institution 'Southern Illinois University System'
Group 'State University Of New York Stony Brook': replacing with canonical Institution 'State University Of New York At Stony Brook'
Group 'Polytechnic Institute Of New York University': replacing with canonical Institution 'New York University'
Group 'Montana State University': replacing with canonical Institution 'Montana State University Bozeman'
Group 'University At Albany Suny': replacing with canonical Institution 'University At Albany Suny'
Group 'Union Institute & University': replacing with canonical Institution 'Union Institute & 

In [243]:
# def get_group_id(row):
#     '''
#     Lookup the institution (normalized by stripping whitespace) and if the institution is not found, then we use affiliation id instead
#     this assigns whichever one is found as a group ID so we can find what school are changing names throughout the years
#     or have slight spelling errors, etc
#     '''
#     inst = row.get("Institution")
#     aff = row.get("AffiliationId")
#     if pd.notna(inst) and normalize(inst) != "":
#         return find(normalize(inst))
#     elif pd.notna(aff) and normalize(aff) != "":
#         return find(normalize(aff))
#     else:
#         return None

# def update_institution_normalized(row):
#     '''
#     Updates institution names so they are the same throughout the years
#     '''
#     group_id = row.get("GroupID")
#     if pd.notna(group_id) and group_id in group_to_canonical and group_to_canonical[group_id]:
#         return group_to_canonical[group_id]
#     else:
#         return row.get("Institution")

# #fuzzy name matching across years
# global_name_mapping = {}  # will hold canonical names across years
# FUZZ_THRESHOLD = 90

# def get_canonical_name(name, mapping, threshold=FUZZ_THRESHOLD):
#     '''
#     find name matches
#     '''
#     # Normalize the name (you might want to use your clean_name function)
#     name_norm = str(name).strip().title()
#     for canonical in mapping.keys():
#         score = fuzz.token_set_ratio(name_norm, canonical)
#         if score >= threshold:
#             return canonical
#     # No match found, so add this name as canonical.
#     mapping[name_norm] = name_norm
#     return name_norm


# normalized_path = "final_scripts\\normalized_dataframes\\"
# state_system_path = f"{absolute_path}{normalized_path}state_systems_validated.csv"
# state_system_df = pd.read_csv(state_system_path)

# for year in years:
#     for board_type in ["_boards", "_double_board"]:
#         file_path = f"{absolute_path}{board_dataframes}{year}{board_type}.csv"
#         df = pd.read_csv(file_path)
#         #add a year column for convenience
#         df = remove_non_samples(df)
#         df["Year"] = year 
        
#         # Add a temporary GroupID column computed via get_group_id.
#         df["GroupID"] = df.apply(get_group_id, axis=1)
#         #standardize the institutions whose names changes slighlty over the years
#         df["Institution"] = df.apply(update_institution_normalized, axis=1)
#         # Optionally drop GroupID now that it has served its purpose.
#         df = df.drop(columns=["GroupID"])
        
#         #normalize affilation id just in case theres some inconsistency for when we map to state systems
#         df["AffiliationId"] = df["AffiliationId"].astype(str).str.strip()
#         state_system_df["AffiliationId"] = state_system_df["AffiliationId"].astype(str).str.strip()
        
#         # Drop any preexisting StateSystem column to avoid duplicates.
#         df = df.drop(columns=["StateSystem"], errors="ignore")
#         # Merge the state system values onto the board dataframe.
#         df = pd.merge(df,
#                       state_system_df[["AffiliationId", "StateSystem"]],
#                       how="left",
#                       on="AffiliationId")
        
#         #fuzzy match names and standardize them
#         df["Name"] = df["Name"].apply(lambda x: get_canonical_name(x, global_name_mapping, FUZZ_THRESHOLD))
        
#         print(df.shape)
#         # Write the normalized DataFrame out to the normalized_dataframes folder.
#         output_file = f"{absolute_path}{normalized_path}{year}{board_type}_normalized.csv"
#         df.to_csv(output_file, index=False)
#         print(f"Normalized data for year {year} ({board_type}) written to {output_file}")


In [244]:
# --- Define helper functions (unchanged) ---
def get_group_id(row):
    # Use the nonempty field from Institution or AffiliationId.
    inst = row.get("Institution")
    aff = row.get("AffiliationId")
    if pd.notna(inst) and normalize(inst) != "":
        return find(normalize(inst))
    elif pd.notna(aff) and normalize(aff) != "":
        return find(normalize(aff))
    else:
        return None

def update_institution_normalized(row):
    group_id = row.get("GroupID")
    if pd.notna(group_id) and group_id in group_to_canonical and group_to_canonical[group_id]:
        return group_to_canonical[group_id]
    else:
        return row.get("Institution")

normalized_path = "final_scripts\\normalized_dataframes\\"
state_system_path = f"{absolute_path}{normalized_path}state_systems_validated.csv"
state_system_df = pd.read_csv(state_system_path)


for year in years:
    # Process both the single board and the double board files.
    for board_type in ["_boards", "_double_board"]:
        file_path = f"{absolute_path}{board_dataframes}{year}{board_type}.csv"
        df = pd.read_csv(file_path)
        
        df = remove_non_samples(df)
        df["Year"] = year  # (optional) in case you want to keep track
        
        # Add a temporary GroupID column computed via get_group_id.
        df["GroupID"] = df.apply(get_group_id, axis=1)
        # Update the Institution column using the canonical name if available.
        df["Institution"] = df.apply(update_institution_normalized, axis=1)
        
        # Optionally drop GroupID now that it has served its purpose.
        df = df.drop(columns=["GroupID"])
        
        # --- NEW: Normalize the AffiliationId columns before merge ---
        df["AffiliationId"] = df["AffiliationId"].astype(str).str.strip()
        state_system_df["AffiliationId"] = state_system_df["AffiliationId"].astype(str).str.strip()
        
        # Merge the state system values onto the board dataframe.
        df = df.drop(columns=["StateSystem"], errors="ignore")
        df = pd.merge(df,
                      state_system_df[["AffiliationId", "StateSystem"]],
                      how="left",
                      on="AffiliationId")
        
        print(df.shape)
        # Write the normalized DataFrame out to the normalized_dataframes folder.
        output_file = f"{absolute_path}{normalized_path}{year}{board_type}_normalized.csv"
        df.to_csv(output_file, index=False)
        print(f"Normalized data for year {year} ({board_type}) written to {output_file}")


(6537, 16)
Normalized data for year 1999 (_boards) written to C:\Users\tykun\OneDrive\Documents\SchoolDocs\VSCodeProjects\connectedData\board_analysis\final_scripts\normalized_dataframes\1999_boards_normalized.csv
(518, 14)
Normalized data for year 1999 (_double_board) written to C:\Users\tykun\OneDrive\Documents\SchoolDocs\VSCodeProjects\connectedData\board_analysis\final_scripts\normalized_dataframes\1999_double_board_normalized.csv
(6647, 16)
Normalized data for year 2000 (_boards) written to C:\Users\tykun\OneDrive\Documents\SchoolDocs\VSCodeProjects\connectedData\board_analysis\final_scripts\normalized_dataframes\2000_boards_normalized.csv
(560, 14)
Normalized data for year 2000 (_double_board) written to C:\Users\tykun\OneDrive\Documents\SchoolDocs\VSCodeProjects\connectedData\board_analysis\final_scripts\normalized_dataframes\2000_double_board_normalized.csv
(6824, 17)
Normalized data for year 2005 (_boards) written to C:\Users\tykun\OneDrive\Documents\SchoolDocs\VSCodeProjects\

In [245]:
# --- Assume group_institutions_by_membership is defined as follows ---
def group_institutions_by_membership(institution_to_members, threshold):
    """
    Given a dictionary: {institution -> set_of_members}, return a list of groups
    (lists) of institutions that have >= `threshold` overlap in membership.
    
    overlap = (size of intersection) / (size of smaller board)
    """
    institutions = list(institution_to_members.keys())
    n = len(institutions)

    # Build adjacency list for institutions with >= threshold overlap
    adjacency = defaultdict(list)
    for i in range(n):
        for j in range(i + 1, n):
            inst_i, inst_j = institutions[i], institutions[j]
            members_i = institution_to_members[inst_i]
            members_j = institution_to_members[inst_j]
            size_i = len(members_i)
            size_j = len(members_j)
            if size_i == 0 or size_j == 0:
                continue
            intersection_size = len(members_i.intersection(members_j))
            smaller_board_size = min(size_i, size_j)
            overlap_ratio = intersection_size / smaller_board_size
            if overlap_ratio >= threshold:
                adjacency[inst_i].append(inst_j)
                adjacency[inst_j].append(inst_i)

    # Find connected components via DFS
    visited = set()
    groups = []
    for inst in institutions:
        if inst not in visited:
            stack = [inst]
            group = []
            while stack:
                current = stack.pop()
                if current not in visited:
                    visited.add(current)
                    group.append(current)
                    for neighbor in adjacency[current]:
                        if neighbor not in visited:
                            stack.append(neighbor)
            groups.append(sorted(group))
    
    # Filter out groups of size 1 (only consider groups with 2+ institutions)
    return [g for g in groups if len(g) > 1]

# --- Set threshold for grouping (adjust as desired) ---
THRESHOLD = 0.7

# --- Process each normalized board DataFrame ---
# (Assuming your normalized files are named like "{year}_boards_normalized.csv")
for year in years:
    board_file = f"{absolute_path}{normalized_path}{year}_boards_normalized.csv"
    df = pd.read_csv(board_file)
    print(f"Year {year} before updating: {df.shape[0]} rows")
    
    # Build a mapping: Institution -> set of board member names.
    # (We only use this mapping to determine which institutions are very similar.)
    institution_to_members = defaultdict(set)
    for _, row in df.iterrows():
        inst = row["Institution"]
        member = row["Name"]  # Assumes the board member name is in the "Name" column.
        institution_to_members[inst].add(member)
    
    # Identify groups of institutions that are nearly identical.
    identical_groups = group_institutions_by_membership(institution_to_members, THRESHOLD)
    print(f"Year {year}: found {len(identical_groups)} identical board groups.")
    
    # Record canonical names for the updated groups.
    updated_institutions = set()
    
    # For each identical group, if any institution in the group has a nonempty StateSystem,
    # then update all rows whose Institution is in that group to have:
    #   Institution = canonical_state  AND  StateSystem = canonical_state.
    for group in identical_groups:
        state_values = []
        for inst in group:
            # Get unique nonempty StateSystem values for rows with this institution.
            vals = df[df["Institution"] == inst]["StateSystem"].dropna().unique()
            vals = [v for v in vals if str(v).strip() != ""]
            state_values.extend(vals)
        if state_values:
            canonical_state = pd.Series(state_values).mode()[0]
            print(f"  Group {group} updated to canonical Institution '{canonical_state}'")
            mask = df["Institution"].isin(group)
            df.loc[mask, "Institution"] = canonical_state
            df.loc[mask, "StateSystem"] = canonical_state
            updated_institutions.add(canonical_state)
    
    # Write the updated (but not yet unioned) DataFrame back.
    df.to_csv(board_file, index=False)
    print(f"Year {year} updated (StateSystem propagation) written to {board_file}")
    
    # --- Now union the boards for those institutions that were updated ---
    if updated_institutions:
        # Select rows for institutions that were part of an identical group.
        df_union = df[df["Institution"].isin(updated_institutions)]
        # Leave the rest intact.
        df_nonunion = df[~df["Institution"].isin(updated_institutions)]
        
        # Group the unioned rows by Institution and aggregate board member names.
        unioned = df_union.groupby("Institution").agg({
            "Name": lambda x: "; ".join(sorted(set(x))),  # union board member names
            "AffiliationId": "first",  # assuming consistency
            "StateSystem": "first",
            "Year": "first"
            # Add other columns as needed.
        }).reset_index()
        unioned["Board_Size"] = unioned["Name"].apply(lambda x: len(x.split("; ")))
        
        # Combine the unioned boards with the nonunioned rows.
        final_df = pd.concat([unioned, df_nonunion], ignore_index=True)
    else:
        # If no institutions were updated, final_df is just the original.
        final_df = df.copy()
    
    # Write the final, unioned DataFrame back to file.
    output_file = f"{absolute_path}{normalized_path}{year}_boards_unioned.csv"
    final_df.to_csv(output_file, index=False)
    print(f"Year {year} unioned board data written to {output_file}")

Year 1999 before updating: 6537 rows
Year 1999: found 7 identical board groups.
  Group ['Arizona State University', 'Northern Arizona University', 'University Of Arizona'] updated to canonical Institution 'Arizona Board of Regents'
  Group ['Iowa State University', 'University Of Iowa'] updated to canonical Institution 'Iowa Board of Regents'
  Group ['Kansas State University', 'University Of Kansas', 'Wichita State University'] updated to canonical Institution 'Kansas Board of Regents'
  Group ['Kent State University', 'Ohio University', 'University Of Akron', 'University Of Cincinnati'] updated to canonical Institution 'University System of Ohio'
  Group ['Mississippi State University', 'University Of Mississippi', 'University Of Southern Mississippi'] updated to canonical Institution 'Mississippi Institutions of Higher Learning'
  Group ['University Of Utah', 'Utah State University'] updated to canonical Institution 'Utah University System'
Year 1999 updated (StateSystem propagatio

In [246]:
# Create a dictionary to hold the set of years for each institution.
institution_years = {}

for year in years:
    file_path = f"{absolute_path}{normalized_path}{year}_boards_normalized.csv"
    # Read the normalized board file for this year.
    df = pd.read_csv(file_path)
    
    # Get the unique institution names from this year.
    for inst in df["Institution"].dropna().unique():
        inst = str(inst).strip()
        if inst == "":
            continue
        if inst not in institution_years:
            institution_years[inst] = set()
        institution_years[inst].add(year)

# Create a set of all years for comparison.
all_years_set = set(years)

# Identify institutions that appear in every year.
institutions_every_year = [inst for inst, yrs in institution_years.items() if yrs == all_years_set]

# Identify institutions that appear only in some years.
institutions_partial = {inst: yrs for inst, yrs in institution_years.items() if yrs != all_years_set}

# Report the results.
print("Number of institutions appearing in every year:", len(institutions_every_year))
print("\nInstitutions that appear in every year:")
for inst in sorted(institutions_every_year):
    print(inst)

print("\nInstitutions that appear in only some years and the years in which they appear:")
for inst, yrs in sorted(institutions_partial.items()):
    print(f"{inst}: {sorted(list(yrs))}")

Number of institutions appearing in every year: 118

Institutions that appear in every year:
Adelphi University
American University
Andrews University
Auburn University
Ball State University
Baylor University
Biola University
Boston College
Boston University
Brandeis University
Brown University
Carnegie Mellon University
Case Western Reserve University
Catholic University Of America
Central Michigan University
Chapman University
Clark Atlanta University
Clark University
Clarkson University
Clemson University
College Of William And Mary
Colorado School Of Mines
Columbia University
Cornell University
Dartmouth College
Drexel University
Duke University
Duquesne University
Eastern Michigan University
Emory University
Florida Institute Of Technology
Fordham University
George Mason University
George Washington University
Georgetown University
Harvard University
Hofstra University
Howard University
Illinois Institute Of Technology
Illinois State University
Indiana State University
James Madis